In [95]:
import matplotlib.pyplot as plt
from shapely import LineString
import segyio
from shapely import Point
import numpy as np
import plotly.graph_objects as go
from shapely.geometry import Polygon


In [40]:
def read_seismic_cube(file_path):
   """
   :param file_path: Путь к файлу
   :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
   а также словарю сейсмотрасс, где ключ это координата пересечения Inlines и Xlines, а значение это список значений сейсмотрасс
   """
   with segyio.open(file_path, 'r') as segyfile:
       x = list(segyfile.ilines)
       y = list(segyfile.xlines)
       z = list(segyfile.samples)
       cube = segyio.tools.cube(segyfile)
       traces = {f"{x[i]}_{y[j]}": cube[i][j][:] for i in range(len(x)) for j in range(len(y))}
   return x, y, z, traces

In [41]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
ilines, xlines, samples, traces = read_seismic_cube(file_path)

In [86]:
# Перевод множества в упорядоченный список в порядке убывания
sorted_coord_x = sorted(ilines, reverse=False)
sorted_coord_y = sorted(xlines, reverse=False)
sorted_coord_z = sorted(samples, reverse=False)
poligon_cub_verh = [(sorted_coord_x[0], sorted_coord_y[0]), (sorted_coord_x[0], sorted_coord_y[-1]), (sorted_coord_x[-1], sorted_coord_y[-1]),(sorted_coord_x[-1], sorted_coord_y[0]), (sorted_coord_x[0], sorted_coord_y[0])]
poligon_cub_sboky = [(sorted_coord_x[0], -1* sorted_coord_z[-1]), (sorted_coord_x[0], -1*sorted_coord_z[0]), (sorted_coord_x[-1], -1*sorted_coord_z[0]),(sorted_coord_x[-1], -1*sorted_coord_z[-1]), (sorted_coord_x[0], -1* sorted_coord_z[-1])]

In [87]:
# проход по всем разломам и составление всех точек в один большой словарь
def process_files(directory):
    files = os.listdir(directory)
    result = {}

    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r') as f:
                data = []
                for line in f:
                    fields = line.split()
                    if len(fields) == 8:
                        x = int(fields[1])
                        y = int(fields[2])
                        z = -int(float(fields[5]))
                        t = int(fields[7])
                        data.append([x, y, z, t])
                result[file] = data

    return result

In [90]:
directory = 'C:/HV/Seismic/datas/Faults_TWT_Charisma_fault_sticks_ASCII'
faults_original = process_files(directory)


In [92]:
"""
Сборка координат в один список попарно
Если точек больше 2, то берется минимальное и максимальное значение по глубинам
Если точка 1, то она не учитывается в разломах
"""
faults_final = {}

for key in faults_original:
    dop_spisok = []
    z_spisok = []
    faults_final[key] = []

    for i in range(len(faults_original[key])):
        if len(dop_spisok) == 0:
            dop_spisok.append(faults_original[key][i])
            z_spisok.append(faults_original[key][i][2])
        elif len(dop_spisok) == 1:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
        elif len(dop_spisok) == 2:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                #faults_final.append(dop_spisok[:])
                faults_final[key].append(dop_spisok[:])
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
        else:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                faults_final[key].append(
                    [dop_spisok[z_spisok.index(min(z_spisok))], dop_spisok[z_spisok.index(max(z_spisok))]])
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])



In [48]:
def line_km(spisok):
    z_min = -2386
    z_max = -2142
    if (-2142 < spisok[0][2]) and (spisok[1][2] < -2386):
        t1 = (z_max - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x1 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t1)
        y1 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t1)
        z1 = z_max

        t2 = (z_min - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x2 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t2)
        y2 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t2)
        z2 = z_min
        return [[x1, y1, z1], [x2, y2, z2], 0]
    elif (-2142 > spisok[0][2]) and (spisok[1][2] < -2386):
        t2 = (z_min - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x2 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t2)
        y2 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t2)
        z2 = z_min
        return [[spisok[0][0], spisok[0][1], spisok[0][2]], [x2, y2, z2], 1]
    elif (-2142 < spisok[0][2]) and (spisok[1][2] > -2386):
        t1 = (z_max - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x1 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t1)
        y1 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t1)
        z1 = z_max
        return [[x1, y1, z1], [spisok[1][0], spisok[1][1], spisok[1][2]], 2]
    else:
        return [[spisok[0][0], spisok[0][1], spisok[0][2]], [spisok[1][0], spisok[1][1], spisok[1][2]], 3]

In [93]:
faults_to_cube_2 = {}
for key in faults_final:
    faults_to_cube_2[key] = []
    for i in range(len(faults_final[key])):
        faults_to_cube_2[key].append(line_km(sorted(faults_final[key][i], reverse=True, key=lambda xx: xx[2])))
maps_of_top = {}
for key in traces:
    maps_of_top[key] = traces[key][0]

In [94]:

def create_polygons(data, axis=(0, 1)):
    """
    Создает полигоны на основе данных, выбирая оси по заданным индексам.
    
    :param data: Словарь с данными о координатах.
    :param axis: Кортеж из двух индексов, определяющих, какие оси использовать (например, (0, 1) для X и Y).
    :return: Словарь с полигонами в виде списка, а координаты в виде списка.
    """
    polygons = {}
    
    for line_name, coordinates_list in data.items():
        # Множество для хранения уникальных точек
        points_set = set()
        
        # Проходим по каждому подсписку в списке координат
        for sublist in coordinates_list:
            # Извлекаем координаты по заданным индексам
            point1 = (sublist[0][axis[0]], sublist[0][axis[1]])  # Используем заданные оси
            point2 = (sublist[1][axis[0]], sublist[1][axis[1]])  # Используем заданные оси
            
            points_set.add(point1)
            points_set.add(point2)
        
        # Преобразуем множество в список и создаем полигон
        unique_points = list(points_set)
        
        
        # Сохранение полигона в словаре с именем линии в качестве ключа
        polygons[line_name] = Polygon(unique_points)          # Координаты в виде списка
        
    
    return polygons




In [120]:
polygons_xy = create_polygons(faults_to_cube_2, axis=(0, 1))  # Используем X и Y
polygons_xz = create_polygons(faults_to_cube_2, axis=(0, 2))  # Используем X и Z

In [115]:
def validate_and_fix_polygon(polygon):
    if not polygon.is_valid:
        #print("Недопустимый полигон:", explain_validity(polygon))
        return polygon.buffer(0)  # Попробуем исправить полигон
    return polygon

In [126]:
def plot_single_polygon(polygon, line_name):
    """
    Визуализирует отдельный полигон с использованием библиотеки Plotly.
    
    :param polygon: Объект Polygon.
    :param line_name: Имя для отображения полигона.
    """
    fig = go.Figure()

    # Извлекаем координаты внешней границы полигона
    x, y = polygon.exterior.xy

    # Преобразуем координаты в списки
    x = list(x)
    y = list(y)

    # Добавляем полигон на график
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name=line_name,
        line=dict(color='blue', width=2)  # Цвет и ширина линии для отдельного полигона
    ))

    return fig

def plot_all_polygons(polygons):
    """
    Визуализирует все полигоны из словаря с использованием библиотеки Plotly.
    
    :param polygons: Словарь с полигонами, где ключи - имена линий, 
                     а значения - объекты Polygon.
    """
    fig = go.Figure()

    for line_name, polygon in polygons.items():
        x, y = polygon.exterior.xy  # Извлекаем координаты внешней границы полигона
        
        # Преобразуем координаты в списки
        x = list(x)
        y = list(y)

        # Добавляем полигон на график
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            name=line_name,
            line=dict(color='red', width=1)  # Цвет и ширина линии для всех полигонов
        ))

    return fig

def plot_verh(poligon_1, slovar, f):

    # Определим полигон A
    polygon_A = Polygon(poligon_1)

    # Определим словарь с полигонами B
    polygons_B = slovar
    
    # Проверяем и исправляем полигоны B
    polygons_B_fixed = {name: validate_and_fix_polygon(poly) for name, poly in polygons_B.items()}

    # Вычисляем площадь полигона A
    area_A = polygon_A.area
    
    # Инициализируем переменную для суммы площадей пересечений
    total_intersection_area = 0
    summ_aria_slovar = 0

    # Вычисляем площади пересечений
    for line_name, polygon in polygons_B_fixed.items():
        intersection = polygon_A.intersection(polygon)
        total_intersection_area += intersection.area
        summ_aria_slovar +=polygon.area

    # Вычисляем процент перекрытия
    if area_A > 0:
        overlap_percentage = (total_intersection_area / summ_aria_slovar) * 100
        
    else:
        overlap_percentage = 0
  

    print(f"Процент перекрытия всех разломов от куба: {overlap_percentage:.2f}%")

    # Визуализируем полигон A
    fig_A = plot_single_polygon(polygon_A, 'Границы Куба')

    # Визуализируем все полигоны из словаря B
    fig_B = plot_all_polygons(polygons_B)

    # Объединяем графики
    fig = fig_A
    for trace in fig_B.data:
        fig.add_trace(trace)
    
    if f == 0:
        # Настройка общего графика
        fig.update_layout(
            title='границы куба и границы разломов, вид сверху',
            xaxis_title='Инлайны',
            yaxis_title='Икслайны',
            showlegend=True
        )
    elif f == 1:
        fig.update_layout(
            title='границы куба и границы разломов, вид сбоку',
            xaxis_title='Инлайны',
            yaxis_title='Глубины',
            showlegend=True
        )
        
# Отображение графика
    fig.show()


In [127]:
plot_verh(poligon_cub_verh, polygons_xy, 0)

Процент перекрытия всех разломов от куба: 100.00%


In [128]:
plot_verh(poligon_cub_sboky, polygons_xz, 1)

Процент перекрытия всех разломов от куба: 100.00%
